<a href="https://colab.research.google.com/github/chousheep/sat_simplification/blob/main/unsloth_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json

file = json.load(open("/content/drive/MyDrive/nlp_lab/marino_mission/batch_finetune.json", "r"))
print(file[1])

{'instruction': 'Translate into simple English.', 'input': 'When Alexa looked at the voluminous e-mail, she was tempted to abridge its contents.', 'output': 'When Alexa looked at the voluminous e-mail, she was tempted to shorten its contents.'}


In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
# For GPU check
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

CUDA available: True
GPU: NVIDIA A100-SXM4-40GB


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"
max_seq_length = 2048  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

==((====))==  Unsloth 2025.9.11: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from datasets import Dataset

def format_prompt(example):
    return f"### Input: {example['input']}\n### Output: {json.dumps(example['output'])}<|endoftext|>"

formatted_data = [format_prompt(item) for item in file]
dataset = Dataset.from_dict({"text": formatted_data})

In [ ]:
# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/outputs", # Changed output directory
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none", # Disable Weights & Biases logging
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/984 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()
# trainer_stats = trainer.train(resume_from_checkpoint = True)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 984 | Num Epochs = 3 | Total steps = 369
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 119,537,664 of 3,940,617,216 (3.03% trained)


Step,Training Loss
25,1.510200
50,1.064200
75,1.015600
100,0.992400
125,0.897900
150,0.630600
175,0.639700
200,0.601400
225,0.589900
250,0.505900


# Test the fine-tuned model with user input

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Get user input
user_input = input("Enter your prompt: ")

# Test prompt
messages = [
    {"role": "user", "content": user_input},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

# Generate attention mask
attention_mask = inputs.new_ones(inputs.shape, dtype=torch.long).to("cuda")

# Generate response
outputs = model.generate(
    input_ids=inputs,
    max_new_tokens=256,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
    top_p=0.9,
    attention_mask=attention_mask,
)

# Decode and print
response = tokenizer.batch_decode(outputs)[0]

# Find the parts of the string
user_start = response.find("<|user|>") + len("<|user|>")
user_end = response.find("<|end|>")
assistant_start = response.find("<|assistant|>") + len("<|assistant|>")
assistant_end = response.find('<endoftext>')

# Extract the input and output
input_text = response[user_start:user_end].strip()
output_text = response[assistant_start:assistant_end].strip().strip('"')

# Format and print
print(f"Output: {output_text}")

Enter your prompt: The new CEO pursued audacious initiatives to save the company from bankruptcy.
Output: The new CEO pursued bold initiatives to save the company from bankruptcy.
